In [71]:
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, Dataset, Subset, ConcatDataset
from pathlib import Path
import numpy as np
from tqdm import tqdm
import abc
import wandb
from torchvision import transforms as T
from byol_pytorch import BYOL
from scipy.spatial.transform import Rotation as R
from PIL import Image
import torchvision
import json
from torch.utils.data import DataLoader
import os

# create abstract Dataset class called StickDataset
import cv2


from utils.r3D_semantic_dataset import load_depth
from utils.metrics import get_act_mean_std
from utils.traverse_data import iter_dir_for_traj_pths


class BaseStickDataset(Dataset, abc.ABC):
    def __init__(self, traj_path, time_skip, time_offset, time_trim):
        super().__init__()
        self.traj_path = Path(traj_path)
        self.time_skip = time_skip
        self.time_offset = time_offset
        self.time_trim = time_trim
        self.img_pth = self.traj_path / "images"
        self.depth_pth = self.traj_path / "depths"
        self.conf_pth = self.traj_path / "confs"
        self.labels_pth = self.traj_path / "labels.json"

        self.labels = json.load(self.labels_pth.open("r"))
        self.img_keys = sorted(self.labels.keys())
        # lable structure: {image_name: {'xyz' : [x,y,z], 'rpy' : [r, p, y], 'gripper': gripper}, ...}

        self.labels = np.array(
            [self.flatten_label(self.labels[k]) for k in self.img_keys]
        )

        # filter using time_skip and time_offset and time_trim. start from time_offset, skip time_skip, and remove last time_trim
        self.labels = self.labels[: -self.time_trim][self.time_offset :: self.time_skip]

        # filter keys using time_skip and time_offset and time_trim. start from time_offset, skip time_skip, and remove last time_trim
        self.img_keys = self.img_keys[: -self.time_trim][
            self.time_offset :: self.time_skip
        ]

    def flatten_label(self, label):
        # flatten label
        xyz = label["xyz"]
        rpy = label["rpy"]
        gripper = label["gripper"]
        return np.concatenate((xyz, rpy, np.array([gripper])))

    def __len__(self):
        return len(self.img_keys)

    def __getitem__(self, idx):
        # not implemented

        raise NotImplementedError


class StickDataset(BaseStickDataset, abc.ABC):
    def __init__(
        self, traj_path, traj_len, time_skip, time_offset, time_trim, traj_skip
    ):
        super().__init__(traj_path, time_skip, time_offset, time_trim)
        self.traj_len = traj_len
        self.traj_skip = traj_skip
        self.reformat_labels(self.labels)
        self.act_metrics = None

    def set_act_metrics(self, act_metrics):
        self.act_metrics = act_metrics

    def reformat_labels(self, labels):
        # reformat labels to be delta xyz, delta rpy, next gripper state
        new_labels = np.zeros_like(labels)
        new_img_keys = []

        for i in range(len(labels) - 1):
            if i == 0:
                current_label = labels[i]
                next_label = labels[i + 1]
            else:
                next_label = labels[i + 1]

            current_matrix = np.eye(4)
            r = R.from_euler("xyz", current_label[3:6], degrees=False)
            current_matrix[:3, :3] = r.as_matrix()
            current_matrix[:3, 3] = current_label[:3]

            next_matrix = np.eye(4)
            r = R.from_euler("xyz", next_label[3:6], degrees=False)
            next_matrix[:3, :3] = r.as_matrix()
            next_matrix[:3, 3] = next_label[:3]

            delta_matrix = np.linalg.inv(current_matrix) @ next_matrix
            delta_xyz = delta_matrix[:3, 3]
            delta_r = R.from_matrix(delta_matrix[:3, :3])
            delta_rpy = delta_r.as_euler("xyz", degrees=False)

            del_gripper = next_label[6] - current_label[6]
            xyz_norm = np.linalg.norm(delta_xyz)
            rpy_norm = np.linalg.norm(delta_r.as_rotvec())

            if xyz_norm < 0.01 and rpy_norm < 0.008 and abs(del_gripper) < 0.05:
                # drop this label and corresponding image_key since the delta is too small (basically the same image)
                continue

            new_labels[i] = np.concatenate(
                (delta_xyz, delta_rpy, np.array([next_label[6]]))
            )
            new_img_keys.append(self.img_keys[i])
            current_label = next_label

        # remove labels with all 0s
        new_labels = new_labels[new_labels.sum(axis=1) != 0]
        assert len(new_labels) == len(new_img_keys)
        self.labels = new_labels
        self.img_keys = new_img_keys

    def load_labels(self, idx):
        # load labels with window size of traj_len, starting from idx and moving window by traj_skip
        labels = self.labels[
            idx * self.traj_skip : idx * self.traj_skip + self.traj_len
        ]
        # normalize labels
        if self.act_metrics is not None:
            labels = (labels - self.act_metrics["mean"].numpy()) / self.act_metrics[
                "std"
            ].numpy()
        return labels

    def get_img_pths(self, idx):
        # get image paths with window size of traj_len, starting from idx and moving window by traj_skip
        img_keys = self.img_keys[
            idx * self.traj_skip : idx * self.traj_skip + self.traj_len
        ]
        img_pths = [self.img_pth / k for k in img_keys]
        return img_pths

    def __len__(self):
        return (len(self.img_keys) - self.traj_len) // self.traj_skip + 1

    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self):
            raise IndexError()
        return None, self.load_labels(idx)


class ImageStickDataset(StickDataset):
    def __init__(
        self,
        traj_path,
        traj_len,
        time_skip,
        time_offset,
        time_trim,
        traj_skip,
        img_size,
        pre_load=False,
        transforms=None,
    ):
        super().__init__(
            traj_path, traj_len, time_skip, time_offset, time_trim, traj_skip
        )
        self.img_size = img_size
        self.pre_load = pre_load
        self.transforms = transforms
        self.preprocess_img_transforms = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(self.img_size),
                torchvision.transforms.ToTensor(),
            ]
        )
        if self.pre_load:
            self.imgs = self.load_imgs()

    def load_imgs(self):
        # load images in uint8 with window size of traj_len, starting from idx and moving window by traj_skip
        imgs = []

        for key in tqdm(self.img_keys):
            img = Image.open(str(self.img_pth / key))
            img = self.preprocess_img_transforms(img)
            imgs.append(img)
        # add a nex axis at the beginning
        imgs = torch.stack(imgs, dim=0)
        return imgs

    def __getitem__(self, idx):
        _, labels = super().__getitem__(idx)

        if self.pre_load:
            imgs = self.imgs[
                idx * self.traj_skip : idx * self.traj_skip + self.traj_len
            ]
        else:
            imgs = []
            for key in self.img_keys[
                idx * self.traj_skip : idx * self.traj_skip + self.traj_len
            ]:
                img = Image.open(str(self.img_pth / key))
                img = self.preprocess_img_transforms(img)
                imgs.append(img)
            # add a nex axis at the beginning
            imgs = torch.stack(imgs, dim=0)

        if self.transforms:
            imgs = self.transforms(imgs)

        return imgs, labels


In [53]:
def get_image_stick_dataset(
    data_path,
    traj_len=1,
    traj_skip=1,
    time_skip=4,
    time_offset=5,
    time_trim=5,
    img_size=224,
    pre_load=True,
    apply_transforms=True,
    val_mask=None,
    mask_texts=None,
    cfg=None,
):
    # add transforms for normalization and converting to float tensor
    if type(data_path) == str:
        data_path = Path(data_path)

    if apply_transforms:
        transforms = torchvision.transforms.Compose(
            [
                torchvision.transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                )
            ]
        )
    else:
        transforms = None

    train_traj_paths, val_traj_paths, test_traj_paths = iter_dir_for_traj_pths(
        data_path, val_mask, mask_texts
    )
    # train_traj_paths = train_traj_paths[:64]
    # val_traj_paths = val_traj_paths[:16]
    # test_traj_paths = test_traj_paths[:16]
    # concatenate all the Datasets for all the trajectories
    train_dataset = ConcatDataset(
        [
            ImageStickDataset(
                traj_path,
                traj_len,
                time_skip,
                time_offset_n,
                time_trim,
                traj_skip,
                img_size,
                pre_load=pre_load,
                transforms=transforms,
            )
            for traj_path, time_offset_n in itertools.product(
                train_traj_paths, [time_offset, time_offset + 2]
            )
        ]
    )

    

    if len(val_traj_paths) > 0:
        val_dataset = ConcatDataset(
            [
                ImageStickDataset(
                    traj_path,
                    traj_len,
                    time_skip,
                    time_offset,
                    time_trim,
                    traj_skip,
                    img_size,
                    pre_load=pre_load,
                    transforms=transforms,
                )
                for traj_path in val_traj_paths
            ]
        )
        
    else:
        val_dataset = None

    if len(test_traj_paths) > 0:
        test_dataset = ConcatDataset(
            [
                ImageStickDataset(
                    traj_path,
                    traj_len,
                    time_skip,
                    time_offset,
                    time_trim,
                    traj_skip,
                    img_size,
                    pre_load=pre_load,
                    transforms=transforms,
                )
                for traj_path in test_traj_paths
            ]
        )
    else:
        test_dataset = None

    return train_dataset, val_dataset, test_dataset

In [54]:
val_mask = {'home': [],
  'env': [],
  'traj': ['2023-04-11--23-20-07_0', '2023-04-11--23-20-22_0']}

apply_transforms = False  ## We apply transforms in byol training so no need to pre apply transforms in the dataset
img_size = [224, 224]

In [45]:
train, val, test = get_image_stick_dataset(data_path='data/Benchmarking_Export/Door_Opening/CDS_Home/Env1', img_size=img_size, val_mask = val_mask, apply_transforms = apply_transforms)

Iterating through:  data/Benchmarking_Export/Door_Opening/CDS_Home/Env1
Total number of trajectories:  24
total number of train trajectories:  24
total number of test trajectories:  0
Total number of val trajectories:  2


100%|██████████| 31/31 [00:00<00:00, 106.77it/s]


## Models

In [6]:
# create an encoder using resnet18 or resnet50

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models


class ImageEncoder(nn.Module):
    #Image encoder takes in an rgb image (or batch of images) as input and outputs N dimensional representation /encoding
    def __init__(
        self, encoder_type="resnet18", pretrained=True, weights=None, cfg=None
    ):
        super(ImageEncoder, self).__init__()
        self.cfg = cfg
        if encoder_type == "resnet18":
            #TODO: Complete the code block here
        elif encoder_type == "resnet50":
            #TODO: Complete the code block here
        else:
            raise ValueError("Unsupported encoder type: {}".format(encoder_type))

        if weights:
            self.encoder.load_state_dict(torch.load(weights))

    def freeze(self):
        #TODO: Complete the code block here

    def forward(self, x):
        return self.encoder(x)


## Trainers

In [49]:
from abc import ABC, abstractmethod


#Base trainer class
class Trainer(ABC):
    def __init__(
        self, model, optimizer, scheduler, device, experiment, save_every, cfg
    ):
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.experiment = experiment
        self.save_every = save_every
        self.cfg = cfg
    


    @abstractmethod
    def train(self, train_dataloader, val_dataloader, test_dataloader, epochs):
        pass

    @abstractmethod
    def validate(self, dataloader):
        pass

    @abstractmethod
    def test(self, dataloader):
        pass

    def save(self, epoch):
        if not os.path.exists("./weights"):
            os.makedirs("./weights")
        torch.save(self.model.state_dict(), f"./weights/{self.experiment}_{epoch}.pth")

    def load(self, path):
        self.model.load_state_dict(torch.load(path))

In [75]:
class BYOLTrainer(Trainer):
    def __init__(
        self, model, optimizer, scheduler, device, experiment, save_every, img_size, cfg = None
    ):
        super().__init__(
            model, optimizer, scheduler, device, experiment, save_every, cfg
        )
        augmentation1 = ??
        augmentation2 = ??
        self.learner = BYOL(??)

    def train(self, train_dataloader, val_dataloader, test_dataloader, epochs):

        self.learner.to(self.device)
        self.model.to(self.device)
        if not os.path.exists("./weights"):
            os.makedirs("./weights")

        for epoch in tqdm(range(epochs)):
            epoch_loss = 0
            for i, (x, _) in enumerate(train_dataloader):
                x = x.to(self.device)
                x = x.squeeze()
               #TODO: Complete the code block here
            
            
               ######

                epoch_loss += loss.item()

            wandb.log(
                {"loss": epoch_loss / len(train_dataloader)}, step=epoch
            )
            if epoch % self.save_every == 0:
                torch.save(
                    self.model.state_dict(),
                    f"./weights/{self.experiment}_{epoch}.pth",
                )


    def validate(self, dataloader):
        pass

    def test(self, dataloader):
        pass


## Training

### Train Encoder using BYOL

In [63]:
experiment = "Tutorial_BYOL"
bs = 512
lr = 3e-4
epochs = 151
num_workers = 4
weight_decay = 1.5e-6
save_every = 50

In [64]:
train_loader = DataLoader(
    train,
    batch_size=bs,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
)

val_loader = DataLoader(
    val,
    batch_size=bs,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
)

In [65]:
model = ??

/ext3/py3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/ext3/py3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [66]:
optimizer = torch.optim.Adam(
    model.parameters(), lr=lr, weight_decay=weight_decay
)

In [67]:
wandb_run = wandb.init(
    project = experiment
)

wandb: wandb version 0.15.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [76]:
byol_trainer = BYOLTrainer(
                model = model, 
                optimizer = optimizer, 
                scheduler = None, 
                device = 'cuda', 
                experiment = experiment, 
                save_every = save_every, 
                img_size = img_size)

In [77]:
byol_trainer.train(
    train_loader, val_loader, None, epochs
)

100%|██████████| 151/151 [06:00<00:00,  2.39s/it]
